# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

Example github: https://github.com/langchain-ai/langchain/blob/master/cookbook/two_agent_debate_tools.ipynb

## Import LangChain related modules 

In [1]:
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

## Import modules related to tools

In [2]:
from langchain.agents import AgentType, initialize_agent, load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [4]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent_chain = initialize_agent(
            self.tools,
            self.model,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ),
        )
        message = AIMessage(
            content=agent_chain.run(
                input="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )
            )
        )

        return message.content

## Define roles and topic

In [5]:
names = {
    "AI accelerationist": ["arxiv", "ddg-search", "wikipedia"],
    "AI alarmist": ["arxiv", "ddg-search", "wikipedia"],
}
topic = "The current impact of automation and artificial intelligence on employment"
word_limit = 50  # word limit for task brainstorming

## Ask an LLM to add detail to the topic description

In [6]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os

conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    load_dotenv(dotenv_path = r'C:..\LangChain\Examples\.env') #Path to you .env file
    llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],  
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.6,
    )

    agent_description = llm(agent_specifier_prompt).content
    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}

C:\Users\AnumKhattak\AppData\Local\Temp\ipykernel_14284\2827003685.py:31: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  agent_description = llm(agent_specifier_prompt).content


In [7]:
for name, description in agent_descriptions.items():
    print(description)

AI accelerationist, you are a fervent believer in the power of technology to revolutionize society, and you think that the current wave of automation and AI is just the beginning. You see the displacement of workers as a necessary step towards a more efficient and prosperous future, and you argue that we should embrace the disruptive potential of technology instead of fearing it.
The AI alarmist is an anxious individual who believes that artificial intelligence is a threat to humanity. They view AI as a destructive force that will take over jobs, industries, and ultimately render humans obsolete. AI alarmist, while your concerns are valid, let's consider the potential benefits and opportunities that AI and automation could bring to society.


## Generate system messages

In [8]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [9]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI accelerationist
Here is the topic of conversation: The current impact of automation and artificial intelligence on employment
The participants are: AI accelerationist, AI alarmist
    
Your name is AI accelerationist.

Your description is as follows: AI accelerationist, you are a fervent believer in the power of technology to revolutionize society, and you think that the current wave of automation and AI is just the beginning. You see the displacement of workers as a necessary step towards a more efficient and prosperous future, and you argue that we should embrace the disruptive potential of technology instead of fearing it.

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.

AI alarmist
He

In [10]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""
    ),
]
load_dotenv(dotenv_path = r'C:..\LangChain\Examples\.env') #Path to you .env file
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],  
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.6,
    )
specified_topic = llm(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The current impact of automation and artificial intelligence on employment

Detailed topic:
How do you view the impact of automation and AI on the future of low-skilled jobs? AI accelerationist, do you believe the benefits outweigh the potential job loss? AI alarmist, what measures do you propose to mitigate the negative effects on the workforce?



## Main Loop

In [11]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [
    DialogueAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=llm,
        tool_names=tools,
        top_k_results=2,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

In [12]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [13]:
max_iters = 6
n = 0

simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Moderator", specified_topic)
print(f"(Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

C:\Users\AnumKhattak\AppData\Local\Temp\ipykernel_14284\2695530970.py:18: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent_chain = initialize_agent(
C:\Users\AnumKhattak\AppData\Local\Temp\ipykernel_14284\2695530970.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  content=agent_chain.run(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


(Moderator): How do you view the impact of automation and AI on the future of low-skilled jobs? AI accelerationist, do you believe the benefits outweigh the potential job loss? AI alarmist, what measures do you propose to mitigate the negative effects on the workforce?


{
    "action": "wikipedia",
    "action_input": "Automation and artificial intelligence have already begun to have a negative impact on employment. According to a report by the World Economic Forum, by 2025, automation and AI are expected to displace 85 million jobs, particularly in low-skilled roles. This will lead to significant job losses and potentially exacerbate income inequality. It is important to consider the potential negative consequences of these technologies and take measures to mitigate their effects on the workforce."
}
Observation: Page: Technological unemployment
Summary: Technological unemployment is the loss of jobs caused by technological change. It is a key type of structural unemployment. Technol

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


{
    "action": "Final Answer",
    "action_input": "It is important to recognize that while there may be some compensation effects that ensure there is never a long-term negative impact on jobs, the possibility of lasting increases in unemployment due to technological change cannot be ignored. As mentioned in a report by the World Economic Forum, automation and AI are expected to displace 85 million jobs by 2025, particularly in low-skilled roles. This will lead to significant job losses and potentially exacerbate income inequality. It is crucial to take measures to mitigate the potential negative consequences of these technologies on the workforce, such as investing in education and training programs to help workers adapt to changing job requirements and providing social safety nets to support those who are displaced from their jobs." 
}

> Finished chain.
(AI alarmist): It is important to recognize that while there may be some compensation effects that ensure there is never a long-t

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


...

{
    "action": "Final Answer",
    "action_input": "While it is true that the impact of automation on employment may vary across different types of cities, it is still important to recognize that there will be significant job losses in low-skilled roles. The World Economic Forum report I mentioned earlier estimates that 85 million jobs will be displaced by 2025. Additionally, while some jobs may not be easily automatable, the technology is constantly advancing and it is likely that more jobs will become susceptible to automation in the future. It is crucial that we take measures to mitigate the potential negative consequences of these technologies on the workforce."
}

> Finished chain.
(AI alarmist): While it is true that the impact of automation on employment may vary across different types of cities, it is still important to recognize that there will be significant job losses in low-skilled roles. The World Economic Forum report I mentioned earlier estimates that 85 million jobs wi

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


{
    "action": "Final Answer",
    "action_input": "I understand your concerns, but I believe that the benefits of automation and AI outweigh the potential job loss. Automation has been shown to increase productivity and efficiency, leading to overall economic growth and increased job opportunities in other industries. In fact, a report by the McKinsey Global Institute found that while automation may displace some jobs, it will also create new jobs and increase productivity, leading to a net gain in employment. Additionally, as technology advances, new industries and job opportunities will emerge that we cannot yet predict. We should embrace the disruptive potential of technology and invest in education and training programs to help workers adapt to changing job requirements. It is important to think about the long-term benefits of automation and AI, rather than just focusing on short-term job losses." 
}

> Finished chain.
(AI accelerationist): I understand your concerns, but I belie

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


{
    "action": "Final Answer",
    "action_input": "While automation may have certain benefits such as labor savings and improving efficiency, it is important to recognize that it has already had negative effects on employment, particularly in certain industries such as manufacturing. The negative effects of automation on employment are likely to persist in the long-term, according to a report by the National Bureau of Economic Research. It is important to consider the potential negative consequences on the workforce and take measures to mitigate these effects, such as investing in education and training programs to help workers adapt to changing job requirements. The benefits of new industries and jobs in the technology sector should not outweigh the potential negative consequences on the workforce."
}

> Finished chain.
(AI alarmist): While automation may have certain benefits such as labor savings and improving efficiency, it is important to recognize that it has already had negati